In [1]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from sklearn.linear_model import ElasticNet

In [2]:
remote_server_uri = "http://localhost:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [3]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [4]:
exp_name = "Elastic_Net_Wine"
mlflow.set_experiment(exp_name)

2022/08/09 12:55:42 INFO mlflow.tracking.fluent: Experiment with name 'Elastic_Net_Wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='Elastic_Net_Wine', tags={}>

In [5]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [6]:
def load_data(data_path):
    data = pd.read_csv(data_path)
    train, test = train_test_split(data)
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, test_x, train_y, test_y

In [7]:
def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    data_path = "datasets/winequality-red.csv"
    train_x, test_x, train_y, test_y = load_data(data_path)
    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        print("Alpha:, l1_ratio:",alpha,l1_ratio)
        print("RMSE:", rmse)
        print("mae:", mae)
        print("r2_score", r2)
        mlflow.log_param(key="alpha",value=alpha)
        mlflow.log_param(key="l1_ratio",value=l1_ratio)
        mlflow.log_metric(key="RMSE",value=rmse)
        mlflow.log_metrics({"mae":mae,"r2_score":r2})
        mlflow.log_artifact(data_path)
        print("Save to {}".format(mlflow.get_artifact_uri()))
        mlflow.sklearn.log_model(lr, "model")
        

In [9]:
train(0.5, 0.5)

2022/08/09 12:57:03 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Alpha:, l1_ratio: 0.5 0.5
RMSE: 0.793164022927685
mae: 0.6271946374319586
r2_score 0.10862644997792625
Save to ./mlruns/1/56aee5e9fe7b4b9899c904a91e34ef26/artifacts


In [10]:
train(0.2, 0.2)

Alpha:, l1_ratio: 0.2 0.2
RMSE: 0.7336400911821402
mae: 0.5643841279275427
r2_score 0.2373946606358417
Save to ./mlruns/1/082ad81d5c0144a6992b23431d9a3acc/artifacts


In [11]:
train(0.1, 0.1)

Alpha:, l1_ratio: 0.1 0.1
RMSE: 0.7128829045893679
mae: 0.5462202174984664
r2_score 0.2799376066653344
Save to ./mlruns/1/7f0fde4ec18246ce854b1ae0c91ecbaa/artifacts
